In [5]:
import copy

def get_points_code(N,r,F):
    points = []
    traces = {} # элементы со следом j
    traces[0] = [a for a in F if a.trace() == 0]
    for B in F:
        if len(points) >= N:
            return points
        if B not in traces[0]:
            c = 2*(B^(r+1)).trace() / B.trace()
            if c not in traces:
                traces[c] = [a for a in F if a.trace() == c]
            for j in traces[c]:
                if len(points) >= N:
                    return points
                points.append([B, j])
    return points

# Автоморфизм кривой
def aut_sigma(xi,i,r):
    return xi * alpha^((r+1)*r^(i-1))

def FGS_encode(N,l,r,e,m,s,F,T2,f):
    n = m*N
    print('Длина кода n =', n)
    q = r^e
    points_code = get_points_code(N,r,F)
    print("Точки кода:", points_code)
    P_inf = T2.places_infinite()[0]
    L1 = (l*P_inf).basis_function_space() # Базис пространства L(l*𝑃∞)
    #print("L(l*𝑃∞):", L1)
    k = len(L1)
    print('Размерность кода k =', k)
    print('Скорость кода R >=', round(float(k/n),5))
    d = N-l/m
    print('Минимальное расстояние кода d >=', d)
    t_min = int((d-1)/2)
    g = T2.genus()
    kappa = ceil((N*(m-s+1) - (k+g-1) + (s+1)*(g/2-1) + 1) / (s+1))
    t = ceil((kappa+l) / (m-s+1))
    t_max = N-t
    print('Число исправляемых ошибок:', t_min, '<= t <=', t_max)
    print('Исходное сообщение f =', f)
    f_RR = 0
    for f_el, L1_el in zip(f, L1):
        f_RR += f_el*L1_el
    PR = PolynomialRing(F, 2, ['x1','x2'])
    f_RR = PR(f_RR)
    #print('Исходное сообщение как элемент L(l*𝑃∞):', f_RR)
    y = [[0 for i in range(0,N)] for j in range(0,m)]
    for i in range(0,m):
        for j in range(0,N):
            x_1 = points_code[j][0]
            x_2 = points_code[j][1]
            for count_sigma in range(0,i):
                x_1 = aut_sigma(x_1,1,r)
                x_2 = aut_sigma(x_2,2,r)
            y[i][j] = f_RR(x_1, x_2)
    return y

def FGS_decode(N,l,e,r,m,s,F,T2,y):
    P_inf = T2.places_infinite()[0]
    L1 = (l*P_inf).basis_function_space() # Базис пространства L(l*𝑃∞)
    #print('L(l*𝑃∞):', L1)
    k = len(L1)
    g = T2.genus()
    D = ceil((N*(m-s+1) - (k+4-1) + (s+1)*(4/2-1) + 1) / (s+1)) # Степенной параметр
    L2 = (D*P_inf).basis_function_space() # Базис пространства L(D*𝑃∞)
    #print('L(D*𝑃∞):', L2)
    L3 = ((l+D)*P_inf).basis_function_space() # Базис пространства L((l+D)*𝑃∞)
    #print('L((l+D)*𝑃∞):', L3)
    points_code = get_points_code(N,r,F)
    count_lines = N*(m-s+1)
    count_columns = len(L3)+len(L2)
    A0_A1_matrix = matrix(F, count_lines, count_columns) # Матрица коэффициентов
    PR = PolynomialRing(F, 2, ['x1','x2'])
    for j in range(0,N):
        x_1 = points_code[j][0]
        x_2 = points_code[j][1]
        for count_sigma in range(0,m-s+1):
            equation_coefficients = []
            for i in range(0,len(L3)):
                L3_i = PR(L3[i])
                equation_coefficients.append(L3_i(x_1, x_2))
            for i in range(0,len(L2)):
                L2_i = PR(L2[i])
                equation_coefficients.append(L2_i(x_1, x_2)*y[count_sigma][j])
            A0_A1_matrix[j*(m-s+1)+count_sigma] = equation_coefficients
            x_1 = aut_sigma(x_1,1,r)
            x_2 = aut_sigma(x_2,2,r)
#     print("Матрица для нахождения A0 и A1:")
    #print(A0_A1_matrix)
    recovered_message_list = []
    for x in A0_A1_matrix.right_kernel().basis():
        #print("Частное решение:",x) 
        _.<Y> = PolynomialRing(F)
        A0_coeff = x[:len(L3)]
        A1_coeff = x[len(L3):]
        A0 = 0
        A1 = 0
        for coeff, L3_elem in zip(A0_coeff, L3):
            A0 += coeff*L3_elem
        for coeff, L2_elem in zip(A1_coeff, L2):
            A1 += coeff*L2_elem
        Q = A0 + A1*Y
    #    print("A0 =", A0)
    #    print("A1 =", A1)
    #    print("Q =", Q)

        count_lines = N
        count_columns = len(L1)
        f_matrix = matrix(F, count_lines, count_columns)
        b = [0]*count_lines
        A0_PR = PR(A0)
        A1_PR = PR(A1)
        for j in range(0,count_lines):
            x_1 = points_code[j][0]
            x_2 = points_code[j][1]
            equation_coefficients = []
            for i in range(0,count_columns):
                L1_i = PR(L1[i])
                equation_coefficients.append(L1_i(x_1, x_2)*A1_PR(x_1,x_2))
            f_matrix[j] = equation_coefficients
            b[j] = -A0_PR(x_1,x_2)
        b_vector = vector(b)
     #   print("Матрица для нахождения исходного сообщения:")
     #   for i in f_matrix:
     #       print(i)
     #   print("Столбец свободных членов:", b_vector)
        try:
            recovered_message = f_matrix.solve_right(b_vector)
            recovered_message_list.append(list(recovered_message))
        except:
            pass
            print("Декодирование невозможно")
#         print("-------------------------")
    return recovered_message_list

    
    

l = 8 # Множитель дивизора (l*𝑃∞)
r = 3 # Характеристика поля констант F
e = 2 # Ступень башни Г-Ш
m = 2 # 1 <= m <= r-1 (число вычисляемых автоморфизмов для каждой точки при кодировании)
s = 1 # s >= 1 (константа декодера)
N = (r-1)*r^e # Кол-во точек кода
# N = 12
F.<alpha> = GF(r^e) # Базовое поле
T1.<x1> = FunctionField(F); _.<Y> = T1[] # Функциональное поле (первая ступень башни Г-Ш)
T2.<x2> = T1.extension(Y^r + Y - (x1^r / (x1^(r-1) + 1))) # Расширение функционального поля (вторая ступень башни Г-Ш)

f = [1, alpha^2, alpha, 0, alpha+1] # Сообщение

# Выбрать рандомное сообщение
# P_inf = T2.places_infinite()[0]
# L1 = (l*P_inf).basis_function_space() # Базис пространства L(l*𝑃∞)
# k = len(L1)
# f = [list(F)[randint(0, len(list(F))-1)] for i in range(0,k)]

print("Кодирование:")
f_encode = FGS_encode(N,l,r,e,m,s,F,T2,f)
print("Кодовое слово:", f_encode)

#Внесём ошибки
# f_encode[0][0] = 2*alpha + 2
# f_encode[1][1] = alpha+1
# f_encode[0][4] = 2*alpha + 1
# f_encode[1][4] = alpha
# f_encode[1][6] = 0
# f_encode[0][6] = 1
# f_encode[0][2] = 2*alpha
# f_encode[1][3] = 2*alpha

print("Кодовое слово с ошибками:", f_encode)
print("------------------------------------------------------")

print("Декодирование:")

recovered_message_list = FGS_decode(N,l,e,r,m,s,F,T2,f_encode)
if (f in recovered_message_list):
    print("Сообщение успешно декодировано!", f)
    print("Список декодированных векторов:", recovered_message_list)
else:
    print("Ошибка", f, recovered_message_list)
    #print(f'Ошибка: {recovered_message_list}')

    
    
    
    
# Для тестирования:

# max_count_error = 3
# for attempt in range(0,500):
#     f = [list(F)[randint(0, len(list(F))-1)] for i in range(0,5)] # Сообщение
#     f_encode = FGS_encode(N,l,r,e,m,s,F,T2,f)
#     #print("Кодовое слово:", f_encode)
#     f_encode_with_errors = copy.deepcopy(f_encode)
#     count_error = 0
#     already_changed = []
#     while count_error < max_count_error:
#         s1 = randint(0,len(f_encode)-1)
#         s2 = randint(0,len(f_encode[s1])-1)
#         random_index_F = randint(0, len(list(F))-1)
#         error = list(F)[random_index_F]
#         if (error != f_encode_with_errors[s1][s2]) and ([s1,s2] not in already_changed):
#             f_encode_with_errors[s1][s2] = error
#             already_changed.append([s1,s2])
#             count_error += 1
#     recovered_message_list = FGS_decode(N,l,e,r,m,s,F,T2,f_encode_with_errors)
#     if (f in recovered_message_list):
#         continue
#         print("Сообщение успешно декодировано!")
#         #print("Список декодированных векторов:", recovered_message_list)
#     else:
#         print("Ошибка", attempt)
#         #print(f'Ошибка: {recovered_message_list}')


Кодирование:
Длина кода n = 36
Точки кода: [[alpha, 2*alpha], [alpha, alpha + 2], [alpha, 1], [2*alpha + 1, 2*alpha], [2*alpha + 1, alpha + 2], [2*alpha + 1, 1], [2, alpha], [2, 2*alpha + 1], [2, 2], [2*alpha, alpha], [2*alpha, 2*alpha + 1], [2*alpha, 2], [alpha + 2, alpha], [alpha + 2, 2*alpha + 1], [alpha + 2, 2], [1, 2*alpha], [1, alpha + 2], [1, 1]]
Размерность кода k = 5
Скорость кода R >= 0.13889
Минимальное расстояние кода d >= 14
Число исправляемых ошибок: 6 <= t <= 6
Исходное сообщение f = [1, alpha + 1, alpha, 0, alpha + 1]
Кодовое слово: [[0, 2*alpha + 1, 2*alpha + 2, 2, alpha + 2, 1, 1, 2, 2*alpha + 2, 2*alpha + 1, alpha + 2, alpha, 2*alpha + 2, 2, 2*alpha + 1, 2*alpha, 2*alpha + 1, alpha + 1], [2*alpha + 1, alpha + 2, alpha, 2*alpha + 2, 2, 2*alpha + 1, 2*alpha, 2*alpha + 1, alpha + 1, 0, 2*alpha + 1, 2*alpha + 2, 2, alpha + 2, 1, 1, 2, 2*alpha + 2]]
Кодовое слово с ошибками: [[0, 2*alpha + 1, 2*alpha + 2, 2, alpha + 2, 1, 1, 2, 2*alpha + 2, 2*alpha + 1, alpha + 2, alpha, 

In [4]:
# Код Рида-Соломона

r = 3
q = r^2
K.<x> = GF(r)[]
F.<a> = GF(q, modulus=x^2+x+2)
n, k = 4, 2
evaluation_points = F.list()[1:n+1]
C = codes.GeneralizedReedSolomonCode(evaluation_points, k)
D = codes.decoders.GRSBerlekampWelchDecoder(C)

message = [a,a+1]
print("Исходное сообщение:", message)
codeword = C.encode(vector(F, message))
print("Кодовое слово:", codeword)

# Внесение ошибок
space = C.ambient_space()
number_errors = D.decoding_radius()
Chan = channels.StaticErrorRateChannel(space, number_errors)
codeword_with_errors = Chan(codeword)
print("Кодовое слово с ошибками", codeword_with_errors)

decoded_codeword = D.decode_to_code(codeword_with_errors)
print("Декодированное кодовое слово:", decoded_codeword)

# recovered_message = list(D.decode_to_message(y)) # То же самое, сразу вернёт сообщение
recovered_message = list(D.connected_encoder().unencode(decoded_codeword))
print("Восстановленное сообщение", recovered_message)

Исходное сообщение: [a, a + 1]
Кодовое слово: (a + 1, 2*a, 1, 2)
Кодовое слово с ошибками (2, 2*a, 1, 2)
Декодированное кодовое слово: (a + 1, 2*a, 1, 2)
Восстановленное сообщение [a, a + 1]


In [29]:
import math
import itertools

# Декодер для (F_p)^n
def BaseFieldDecoder(y,e0):
    y = vector(y)
    distance = int(e0)
    delta_list = [i for i in range(-distance, distance+1)] # Диапазон значений для построение списка delta_vector'ов ([-2,-1,0,1,2])

    code_words_list = []
    for delta in itertools.product(delta_list, repeat=len(y)): # Все перестановки delta_vector'ов
        delta_vector = vector(delta)
        v = y + delta_vector
        if ((y-v).norm()) <= e0:
            code_words_list.append(v)

    return code_words_list

# Декодер кода Рида-Соломона
def RSDecoder(n,k,F,codeword_with_errors):
    evaluation_points = F.list()[1:n+1]
    C = codes.GeneralizedReedSolomonCode(evaluation_points, k)
    D = codes.decoders.GRSBerlekampWelchDecoder(C)
    y = vector(F, codeword_with_errors)
    try:
        decoded_codeword = D.decode_to_code(y)
        decoded_codeword_int = list(map(int, decoded_codeword))
        return [decoded_codeword_int]
    except:
        return []
    
# Декодирование решётки конструкции D
def LatticeDecoder(y,i,e0,p):
    ei = e2 = p^i*e0
    w = [round(j) % p for j in y]
    if (i == 0):
        L = BaseFieldDecoder(w, e0)
    else:
        L = RSDecoder(n,k,F,w)
    R_c = []
    for c in L:
        if i == 0:
            res = [b + (round((a-b) / p) * p) for a,b in zip(y, c)]
            R_c.append(res)
        else:
            y_new = [(a-b) / p for a,b in zip(y, c)]
            for v in LatticeDecoder(y_new,i-1,e0,p):   
                rez = [a + p*b for a,b in zip(c,v)]
                if ((vector(y)-vector(rez)).norm() <= ei):
                    R_c.append(rez)
    return R_c

p = 5
F = GF(p)
n, k = 4, 2
i = 1
e0 = 2
y = [8.2, 4.4, 5.9, 6.3]

vectors = LatticeDecoder(y,i,e0,p)
print(vectors)

[[3, -1, 0, 6], [3, -1, 5, 1], [3, -1, 5, 6], [3, -1, 5, 11], [3, -1, 10, 6], [3, -1, 10, 11], [3, 4, 0, 1], [3, 4, 0, 6], [3, 4, 0, 11], [3, 4, 5, 1], [3, 4, 5, 6], [3, 4, 5, 11], [3, 4, 10, 1], [3, 4, 10, 6], [3, 4, 10, 11], [3, 9, 0, 6], [3, 9, 5, 1], [3, 9, 5, 6], [3, 9, 5, 11], [3, 9, 10, 1], [3, 9, 10, 6], [3, 9, 10, 11], [8, -1, 0, 1], [8, -1, 0, 6], [8, -1, 0, 11], [8, -1, 5, 1], [8, -1, 5, 6], [8, -1, 5, 11], [8, -1, 10, 1], [8, -1, 10, 6], [8, -1, 10, 11], [8, 4, 0, 1], [8, 4, 0, 6], [8, 4, 0, 11], [8, 4, 5, 1], [8, 4, 5, 6], [8, 4, 5, 11], [8, 4, 5, 16], [8, 4, 10, 1], [8, 4, 10, 6], [8, 4, 10, 11], [8, 4, 15, 6], [8, 9, 0, 1], [8, 9, 0, 6], [8, 9, 0, 11], [8, 9, 5, 1], [8, 9, 5, 6], [8, 9, 5, 11], [8, 9, 10, 1], [8, 9, 10, 6], [8, 9, 10, 11], [8, 14, 5, 6], [13, -1, 0, 6], [13, -1, 5, 1], [13, -1, 5, 6], [13, -1, 5, 11], [13, -1, 10, 1], [13, -1, 10, 6], [13, -1, 10, 11], [13, 4, 0, 1], [13, 4, 0, 6], [13, 4, 0, 11], [13, 4, 5, 1], [13, 4, 5, 6], [13, 4, 5, 11], [13, 4, 10,